# Qubit Counts

The number of qubits is an important cost for running a quantum algorithm. The provided `QubitCounts()` cost key can efficiently estimate the qubit count of even large-scale algorithms by exploiting the hierarchical structure of bloq decomposition.


The number of qubits is bounded from below by the number of qubits implied by the signature.
If a bloq has no callees, the size implied by the signature will be returned. Otherwise,
`QubitCounts()` will try to compute the number of qubits by inspecting the decomposition.

In the decomposition, each (sub)bloq is considered to be executed sequentially. The "width"
of the circuit (i.e. the number of qubits) at each sequence point is the number of qubits
required by the subbloq (computed recursively) plus any "bystander" idling wires.

This is an estimate for the number of qubits required by an algorithm. Specifically:
 - Bloqs are assumed to be executed sequentially, minimizing the number of qubits potentially
   at the expense of greater circuit depth or execution time.
 - We do not consider "tetris-ing" subbloqs. In a decomposition, each subbloq is assumed
   to be using all of its qubits for the duration of its execution. This could potentially
   overestimate the total number of qubits.

This Min-Max style estimate can provide a good balance between accuracy and scalability
of the accounting. To fully account for each qubit and manage space-vs-time trade-offs,
you must comprehensively decompose your algorithm to a `cirq.Circuit` of basic gates and
use a `cirq.QubitManager` to manage trade-offs. This may be computationally expensive for
large algorithms.

In [ ]:
import sympy

from qualtran.drawing import show_bloq

from qualtran.bloqs.for_testing.interior_alloc import InteriorAlloc
from qualtran.resource_counting import get_cost_value, query_costs, QubitCount

For illustrative purposes, we use a bloq that has two $n$ bit registers, but allocates an additional $n$ bit register as part of its decomposition. Looking purely at the signature, you would conclude that the bloq uses $2n$ qubits; but by looking at the decomposition we can see that at its maximum circuit width it uses $3n$ qubits. 

In [ ]:
n = sympy.Symbol('n', positive=True, integer=True)
bloq = InteriorAlloc(n=n)
show_bloq(bloq)
show_bloq(bloq.decompose_bloq(), 'musical_score')

In [ ]:
get_cost_value(bloq, QubitCount())

In [ ]:
from qualtran.drawing import GraphvizCallGraph

g, _ = bloq.call_graph()
costs = query_costs(bloq, [QubitCount()])
GraphvizCallGraph(g, costs).get_svg()

In [ ]:
from qualtran.bloqs.state_preparation import PrepareUniformSuperposition
from qualtran.resource_counting import get_cost_value, QubitCount
from qualtran.drawing import show_bloqs

bloq = PrepareUniformSuperposition(n=3,cvs=(1,))
cbloq = bloq.decompose_bloq()
cbloq_flat = cbloq.flatten()
print(get_cost_value(bloq, QubitCount()))
print(get_cost_value(cbloq, QubitCount()))
print(get_cost_value(cbloq_flat, QubitCount()))
show_bloq(bloq, 'musical_score')
show_bloq(cbloq, 'musical_score')
show_bloq(cbloq_flat, 'musical_score')

In [ ]:
from qualtran.drawing import get_musical_score_data, draw_musical_score
draw_musical_score(get_musical_score_data(cbloq_flat), max_width=100)

In [ ]:
from qualtran.resource_counting._qubit_counts import _cbloq_max_width

In [ ]:
_cbloq_max_width(cbloq_flat._binst_graph)

In [ ]:
from qualtran.resource_counting import get_bloq_callee_counts
get_bloq_callee_counts(cbloq_flat)